In [1]:
import pandas as pd
import re
import math



In [2]:
data = pd.read_csv("result/mc_gpt_35.csv",index_col=0)
data.head(5)

,ind,id,activity_label,ctx_a,ctx_b,ctx,dataset,split_type,ending_options,label,first_part,slot_question,slot_answer,pred_answer
0,19073,31450,Personal Care and Style,[header] How to get perfect eyebrows [title] F...,NaN,[header] How to get perfect eyebrows [title] F...,wikihow,zeroshot,['Measure the length of your inner brow and th...,3,[header] How to get perfect eyebrows [title] F...,[header] How to get perfect eyebrows [title] F...,It should create a slight curve out the corner...,option1: Use a ruler to measure the distance b...
1,20017,33062,Personal Care and Style,[header] How to make french knot easy way hair...,NaN,[header] How to make french knot easy way hair...,wikihow,zeroshot,"[""For this reason, french knot style hair will...",3,[header] How to make french knot easy way hair...,[header] How to make french knot easy way hair...,"For this reason, french knot style hair will b...",option3: [substeps] Gather all of your hair in...
2,34560,v_pHAIrx_cRFU,Sharpening knives,The person puts a new belt on the machine. The...,the person,The person puts a new belt on the machine. The...,activitynet,zeroshot,"['puts the sword back on the machine.', 'remov...",3,The person puts a new belt on the machine,The person puts a new belt on the machine. The...,puts the sword back on the machine.,option3: puts on a third belt.
3,41427,68173,Family Life,[header] How to plan your own funeral [title] ...,NaN,[header] How to plan your own funeral [title] ...,wikihow,zeroshot,['[title] Prepare for your funeral by creating...,3,[header] How to plan your own funeral [title] ...,[header] How to plan your own funeral [title] ...,It might be something simple like the experien...,[title] Consider the memorial service options ...
4,9755,16080,Health,[header] How to buy an exercise bike [title] E...,NaN,[header] How to buy an exercise bike [title] E...,wikihow,indomain,['You will want to check with your trainer bef...,3,[header] How to buy an exercise bike [title] E...,[header] How to buy an exercise bike [title] E...,They get you working less because you are doin...,option1: Evaluate the available space in your ...


In [12]:
from rouge_score import rouge_scorer
scorer = rouge_scorer.RougeScorer(['rouge1','rouge2','rougeL'], use_stemmer=True)

In [24]:
f_score = 0
acc = 0
slot_ans_list = []
answer_list = []
for index,row in data.iterrows():
    answer = row['slot_answer']
    slot_answer = row['pred_answer']
    if (type(slot_answer)!=type("asd")):
        continue
    if "Answer: " in slot_answer:
        if slot_answer.split("Answer: ")[1] != "":
            slot_answer = slot_answer.split("Answer: ")[1]
    elif "(" in slot_answer:
        pattern = r'\((.*?)\)'
        matches = re.findall(pattern, slot_answer)
        if len(matches) !=0:
            slot_answer = matches[0]
        else:
            print(row['sentence'])
    rouge_score_tmp = scorer.score(slot_answer,answer)
    f_score_tmp = rouge_score_tmp['rougeL'].fmeasure
    f_score += f_score_tmp
    print(answer,"|",slot_answer)
    slot_ans_list.append(slot_answer)
    answer_list.append(answer)
    if slot_answer.lower() == answer.lower():
        acc += 1
print("Acc: ",acc*1.0/data.shape[0])
print("Rouge-l F1: ",f_score*1.0/data.shape[0])
    

Sarah | Ian
Maria | Tom
blanket | wine
toaster | small
Hunter | someone
Maria | A veterinarian.
house | apartment
home | childhood home
Natalie | Jennifer
bank | 
bakery | Joe immediately went to the pawn shop before the bank because the pawn shop had a substantial supply of what he wanted.
story | book
class | Reading
library | library
cafe | cafe
Natalie | Luis
Michael | Ian
Leslie | Michael
me
Neil | Leslie
Ian
Eric | Jacob
pudding | food coloring
Benjamin | Olivia
Cynthia | Someone else
strength | strength
work | recover
Betty | 
Derrick | Leslie
sizes | yesterday
Benjamin | option1
Jessica | 
pencils | clips
Rachel | 
Mary | They
Kyle | they
Jessica | Irene
bathroom | room
Joel | Olivia
Emily | her boss
Angela | Sheila
Donald | Ian
Brett | Brian
Ian
eyes | eyes
dress | dress
hat | Dress.
Patricia | You
William | Ian
Craig | Ian
Angela | Samantha
Hunter | Ian
Amy | - Tanya
- I
crack | glass
brush | brush
comics | comics
Jennifer | Ian
Natalie | Jason
Matthew | Ian
food | water
Rand

In [ ]:
# data['slot_answer'] = slot_ans_list
# data['answer'] = answer_list
# data.to_csv("result/mc_gpt_4.csv")